code 変更箇所の記載
- 引数
- メトリック記録

local 環境でのテスト

azure machine learning training

In [12]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [13]:
ws = Workspace.from_config()

In [14]:
dataset = Dataset.get_by_name(ws, name='cifar10')

In [15]:
experiment = Experiment(workspace=ws, name='day1-experiment-data')

config = ScriptRunConfig(source_directory='./code/pytorch-cloud',
                         script='train.py',
                         compute_target='cpu-cluster',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_mount(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])



In [16]:
env = Environment.get(ws, "pytorch-env")

In [17]:
config.run_config.environment = env

In [18]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

ps = RandomParameterSampling(
    {
        '--learning_rate': loguniform(-6, -1),
        '--momentum': loguniform(-6, -1),
    }
)

In [19]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [20]:

hyperdrive_config = HyperDriveConfig(run_config=config,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='train_loss',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [21]:
# set up pytorch environment


run = experiment.submit(hyperdrive_config)
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…